In [13]:
import wrf_management.run_utilities
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import wrf_management.ungrib as un
importlib.reload(un)
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml
import subprocess as su
import pathlib

In [14]:
print(gc.RUN_NAME)
job = 'ungrib_avgtsfc'
ungrib_source_dirs = ['ungrib_surf', 'ungrib_press']

real = True
hours = 24

LIST_S_LINKS = [
    'avg_tsfc.exe',
#     'link_grib.csh',
#     'ungrib'
]

LIST_H_LINKS = [
#     'Vtable',
    'env_WRFv4.bash'
]

run_2019_02_20


In [15]:
#con = sq.connect(gc.PATH_DB)
gc.PATH_DB

'/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/db_folder/wrf_man.sqlite'

In [16]:
importlib.reload(un)
run_row = wrf_management.run_utilities.get_run_row()
print(run_row)

index                             1
run_name             run_2019_02_20
geogrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
parent_run                     None
comments                       None
Name: 0, dtype: object


In [17]:
job_row = wrf_management.run_utilities.get_next_row(job=job, i_max=10)
print(job_row)

date              2018-05-18 00:00:00
ungrib_surf                       200
ungrib_press                      200
ungrib_avgtsfc                      5
metgrid                             5
real                                0
Name: 0, dtype: object


In [18]:
if real:
    wrf_management.run_utilities.update_run_table(val=job_row[job] + 1,
                                                  job=job,
                                                  date=job_row['date']
                                                  )

job_path = wrf_management.run_utilities.getmk_job_path(run_row, job_row, job)
print(job_path)

wrf_management.run_utilities.rm_if_path_exists(
    os.path.join(job_path, 'avg_tsfc.log')
)

/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2018_05_18/ungrib_avgtsfc


In [19]:
conf_path = wrf_management.run_utilities.get_conf_path(run_row)
print(conf_path)

/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/config_dir/run_2019_02_20


In [20]:
importlib.reload(un)
name_list = un.skim_namelist_copy_avgtsfc(
    conf_path, job_path, date=job_row.date, prefix=job, hours=24
)
pd.DataFrame(name_list)

,share,metgrid
end_date,"[2018-05-19_00:00:00, 2018-05-19_00:00:00, 201...",NaN
fg_name,NaN,"[ungrib_surf, ungrib_press]"
interval_seconds,21600,NaN
io_form_geogrid,2,NaN
io_form_metgrid,NaN,2
max_dom,4,NaN
start_date,"[2018-05-18_00:00:00, 2018-05-18_00:00:00, 201...",NaN
wrf_core,ARW,NaN


In [21]:

importlib.reload(un)
un.link_grub_files(ungrib_prefixes=ungrib_source_dirs, job_path=job_path)

In [22]:
# if gc.ID=='taito_login':
wrf_management.run_utilities.copy_hard_links(conf_path, job_path, LIST_H_LINKS)
wrf_management.run_utilities.copy_soft_links(
    os.path.join(gc.PATH_WPS,'util'),
    job_path,LIST_S_LINKS)
    


env_WRFv4.bash
unilinking
avg_tsfc.exe


In [23]:
run_script = \
    """#!/bin/bash
        
cd {job_path}
source ./env_WRFv4.bash 
srun -t10 -p serial --mem 1000 -J'a{date}' ./avg_tsfc.exe > avg_tsfc.log
exit $?
    """.format(job_path=job_path, date = job_row.date)
print(run_script)
bs_path = os.path.join(job_path, 'run_me.sh')
bs_file = open(bs_path, 'w')
bs_file.write(run_script)
bs_file.close()

#!/bin/bash
        
cd /wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2018_05_18/ungrib_avgtsfc
source ./env_WRFv4.bash 
srun -t10 -p serial --mem 1000 -J'a2018-05-18 00:00:00' ./avg_tsfc.exe > avg_tsfc.log
exit $?
    


In [24]:
if gc.ID == 'taito_login':
    res = su.run(['/bin/bash', bs_path], stdout=su.PIPE, stderr=su.PIPE)

print(res.stdout)
print(res.stderr)
if gc.ID == 'taito_login' and res.returncode == 0:
    wrf_management.run_utilities.update_run_table(val=100,
                                                  job=job,
                                                  date=job_row['date']
                                                  )

b''
b'srun: job 41577624 queued and waiting for resources\nsrun: job 41577624 has been allocated resources\n'
